# Créer et exécuter un pipeline RAG local à partir de zéro

L'objectif de ce notebook est de créer un pipeline RAG (Retrieval Augmented Generation) à partir de zéro et de le faire fonctionner sur un GPU local.

Plus précisément, nous aimerions pouvoir ouvrir un fichier PDF, lui poser des questions (requêtes) et y répondre par un Large Language Model (LLM).

Il existe des frameworks qui reproduisent ce type de flux de travail, notamment [LlamaIndex](https://www.llamaindex.ai/) et [LangChain](https://www.langchain.com/). Cependant, l'objectif de construire à partir de scratch, c'est pouvoir inspecter et personnaliser toutes les pièces.

## Pourquoi RAG ?

L'objectif principal de RAG est d'améliorer le rendement de génération des LLM.

Deux améliorations principales peuvent être considérées comme :
1. **Prévenir les hallucinations** - Les LLM sont incroyables mais ils sont sujets à des hallucinations potentielles, comme générer quelque chose qui *semble* correct mais ne l'est pas. Les pipelines RAG peuvent aider les LLM à générer davantage de résultats factuels en leur fournissant des entrées factuelles (récupérées). Et même si la réponse générée à partir d'un pipeline RAG ne semble pas correcte, grâce à la récupération, vous avez également accès aux sources d'où elle provient.
2. **Travailler avec des données personnalisées** - De nombreux LLM de base sont formés avec des données textuelles à l'échelle Internet. Cela signifie qu’ils ont une grande capacité à modéliser le langage, mais qu’ils manquent souvent de connaissances spécifiques. Les systèmes RAG peuvent fournir aux LLM des données spécifiques à un domaine telles que des informations médicales ou de la documentation d'entreprise et ainsi personnaliser leurs sorties pour répondre à des cas d'utilisation spécifiques.

Les auteurs de l’article original du RAG mentionné ci-dessus ont souligné ces deux points dans leur discussion.

> Ce travail offre plusieurs avantages sociétaux positifs par rapport aux travaux antérieurs : le fait qu'il soit plus
fortement ancré dans des connaissances factuelles réelles (dans ce cas Wikipédia), le rend moins « halluciné »
avec des générations plus factuelles et offre plus de contrôle et d’interprétabilité. RAG pourrait être
employé dans une grande variété de scénarios bénéficiant directement à la société, par exemple en la dotant
avec un index médical et en lui posant des questions ouvertes sur ce sujet, ou en aidant les gens à être plus
efficaces dans leur travail.

RAG peut également être une solution beaucoup plus rapide à mettre en œuvre que d’affiner un LLM sur des données spécifiques.

Pourquoi local ?
Confidentialité, rapidité, coût.

Exécuter localement signifie que vous utilisez votre propre matériel.

Du point de vue de la confidentialité, cela signifie que vous n'avez pas besoin d'envoyer de données potentiellement sensibles à une API.

Du point de vue de la vitesse, cela signifie que vous n'aurez pas nécessairement à attendre une file d'attente d'API ou un temps d'arrêt ; si votre matériel est en cours d'exécution, le pipeline peut s'exécuter.

Et du point de vue des coûts, fonctionner sur votre propre matériel entraîne souvent un coût de départ plus élevé, mais peu ou pas de frais par la suite.

En termes de performances, les API LLM peuvent toujours fonctionner mieux qu'un modèle open source exécuté localement sur des tâches générales, mais il existe de plus en plus d'exemples de modèles plus petits et ciblés surpassant les modèles plus grands.

## Termes clés

| Term | Description |
| ----- | ----- | 
| **Jeton** | Un morceau de texte sous-mot. Par exemple, « Bonjour tout le monde ! » pourrait être divisé en ["hello", ",", "world", "!"]. Un jeton peut être un mot entier,<br> une partie d'un mot ou un groupe de caractères de ponctuation. 1 jeton ~= 4 caractères en anglais, 100 jetons ~= 75 mots.<br> Le texte est divisé en jetons avant d'être transmis à un LLM. |
| **Embedding** | Une représentation numérique apprise d’une donnée. Par exemple, une phrase de texte pourrait être représentée par un vecteur avec<br> 768 valeurs. Des morceaux de texte similaires (dans leur sens) auront idéalement des valeurs similaires. |
| **Embedding model** | Un modèle conçu pour accepter des données d'entrée et produire une représentation numérique. Par exemple, un modèle d'incorporation de texte peut prendre 384 <br>jetons de texte et le transformer en un vecteur de taille 768. Un modèle d'incorporation peut et est souvent différent d'un modèle LLM. |
| **Similarity search/vector search** | Similarity search/vector la recherche vise à trouver deux vecteurs proches l’un de l’autre dans un espace de haute dimension. Par exemple, deux morceaux de texte similaire transmis via un modèle d'intégration devraient avoir un score de similarité élevé, tandis que deux morceaux de texte sur<br> des sujets différents auront un score de similarité plus faible. Les mesures courantes du score de similarité sont la similarité du produit scalaire et du cosinus. |
| **Large Language Model (LLM)** | Un modèle qui a été entraîné pour représenter numériquement les modèles dans le texte. Un LLM génératif continuera une séquence lorsqu'on lui donnera une séquence. <br>Par exemple, étant donné une séquence du texte « bonjour tout le monde ! », un LLM génératif peut produire « nous allons construire un pipeline RAG aujourd'hui ! ».<br> Cette génération sera fortement dépendante de la formation données et invite.|
| **LLM context window** | Le nombre de jetons qu'un LLM peut accepter en entrée. Par exemple, depuis mars 2024, GPT-4 dispose d'une fenêtre contextuelle par défaut de 32 000 jetons<br> (environ 96 pages de texte), mais peut aller jusqu'à 128 000 si nécessaire. Un récent LLM open source de Google, Gemma (mars 2024) a une fenêtre contextuelle<br> de 8 192 jetons (environ 24 pages de texte). Une fenêtre de contexte plus élevée signifie qu'un LLM peut accepter des informations plus pertinentes<br> pour faciliter une requête. Par exemple, dans un pipeline RAG, si un modèle dispose d'une fenêtre contextuelle plus grande, il peut accepter davantage d'éléments de référence<br> du système de récupération pour faciliter sa génération.|
| **Prompt** | Un terme courant pour décrire l'entrée dans un LLM génératif. L'idée de « [prompt Engineering](https://en.wikipedia.org/wiki/Prompt_engineering) » est de structurer une entrée basée sur du texte<br> (ou potentiellement également basée sur des images) dans un LLM génératif dans un manière spécifique afin que la sortie générée soit idéale. Cette technique est<br>possible grâce à la capacité d'un LLM à apprendre en contexte, car il est capable d'utiliser sa représentation du langage pour décomposer l'invite et reconnaître ce que peut être un résultat approprié (remarque : le résultat des LLM est probable, c'est pourquoi des termes tels que « peut produire » sont utilisés).| 




## Ce que nous allons construire

Nous allons créer un pipeline RAG qui nous permet de discuter avec un document PDF, en particulier des pdfs sur les programmes des lycées au Cameroun.

Vous pourriez appeler notre projet EduChat !

Nous écrirons le code dans :
1. Ouvrez un document PDF (vous pouvez utiliser presque n'importe quel PDF ici).
2. Formatez le texte du manuel PDF pour le préparer à un modèle d'Embedding(ce processus est connu sous le nom de fractionnement/blocage de texte).
3. Intégrez tous les morceaux de texte dans le manuel et transformez-les en représentation numérique que nous pourrons stocker pour plus tard.
4. Créez un système de récupération qui utilise la recherche vectorielle pour trouver des morceaux de texte pertinents en fonction d'une requête.
5. Créez une invite qui intègre les morceaux de texte récupérés.
6. Générez une réponse à une requête basée sur des passages du manuel.

Les étapes ci-dessus peuvent être divisées en deux sections principales :
1. Prétraitement/création d'Embedidng des document (étapes 1 à 3).
2. Recherchez et répondez (étapes 4 à 6).

Et c'est la structure que nous suivrons.

C'est similaire au flux de travail décrit sur le blog NVIDIA qui [détaille un pipeline RAG local](https://developer.nvidia.com/blog/rag-101-demystifying-retrieval-augmented-generation-pipelines/).

<img src="https://github.com/mrdbourke/simple-local-rag/blob/main/images/simple-local-rag-workflow-flowchart.png?raw=true" alt="organigramme d'un local Flux de travail RAG" />

## 1. Traitement de documents/textes et création d'Embeddings

Ingrédients:
* Document PDF au choix.
* Modèle d'intégration au choix.

Mesures:
1. Importez un document PDF.
2. Traitez le texte pour l'intégrer (par exemple, divisé en morceaux de phrases).
3. Intégrez des morceaux de texte avec le modèle d'intégration.
4. Enregistrez les intégrations dans un fichier pour une utilisation ultérieure (les intégrations seront stockées dans un fichier pendant de nombreuses années ou jusqu'à ce que vous perdiez votre disque dur).

Importer les document PDF
Cela fonctionnera avec de nombreux autres types de documents.

Il existe plusieurs bibliothèques pour ouvrir des PDF avec Python mais j'ai trouvé que PyMuPDF fonctionne plutôt bien dans de nombreux cas.


In [2]:
import os
from tqdm.auto import tqdm  # Pour afficher une barre de progression
import fitz  # PyMuPDF, pour lire les fichiers PDF

# Fonction pour formater le texte
def formater_texte(texte: str) -> str:
    """Effectue un léger formatage du texte."""
    texte_nettoye = texte.replace("\n", " ").strip()  # Remplace les sauts de ligne par des espaces et supprime les espaces inutiles
    return texte_nettoye

# Fonction pour ouvrir et lire un fichier PDF
def ouvrir_et_lire_pdf(chemin_pdf: str) -> list[dict]:
    """
    Ouvre un fichier PDF, lit son contenu page par page et collecte des statistiques.

    Paramètres :
        chemin_pdf (str) : Chemin d'accès au fichier PDF à ouvrir et lire.

    Retourne :
        list[dict] : Une liste de dictionnaires contenant le numéro de page,
        le nombre de caractères, le nombre de mots, le nombre de phrases,
        le nombre estimé de tokens et le texte extrait pour chaque page.
    """
    doc = fitz.open(chemin_pdf)  # Ouvre le document PDF
    pages_et_textes = []
    for numero_page, page in tqdm(enumerate(doc), desc=f"Traitement de {os.path.basename(chemin_pdf)}"):  # Parcourt les pages
        texte = page.get_text()  # Extrait le texte de la page
        texte = formater_texte(texte)  # Nettoie le texte
        pages_et_textes.append({
            "numero_page": numero_page + 1,  # Les numéros de page commencent à 1
            "nb_caracteres": len(texte),
            "nb_mots": len(texte.split(" ")),
            "nb_phrases": len(texte.split(". ")),
            "nb_tokens_estime": len(texte) / 4,  # Estimation des tokens (1 token ≈ 4 caractères)
            "texte": texte
        })
    return pages_et_textes

# Dossier contenant les fichiers PDF locaux
dossier_pdf = "data"  # Modifiez ceci si votre dossier a un autre nom

# Vérifie si le dossier existe
if not os.path.exists(dossier_pdf):
    raise FileNotFoundError(f"Le dossier '{dossier_pdf}' n'existe pas.")

# Liste tous les fichiers PDF dans le dossier
fichiers_pdf = [os.path.join(dossier_pdf, f) for f in os.listdir(dossier_pdf) if f.endswith(".pdf")]

# Traite tous les fichiers PDF du dossier
contenu_tous_les_pdfs = {}

for fichier_pdf in fichiers_pdf:
    print(f"Lecture du fichier : {fichier_pdf}")
    contenu_pdf = ouvrir_et_lire_pdf(fichier_pdf)  # Lit le contenu du PDF
    contenu_tous_les_pdfs[os.path.basename(fichier_pdf)] = contenu_pdf  # Stocke le contenu avec le nom du fichier comme clé

# Exemple : Affiche les 2 premières pages de chaque PDF traité
for nom_pdf, contenu in contenu_tous_les_pdfs.items():
    print(f"\n--- {nom_pdf} ---")
    for page in contenu[:2]:  # Affiche uniquement les 2 premières pages
        print(f"Page {page['numero_page']} :")
        print(page['texte'])
        print("\n")


Lecture du fichier : data/Physique-Chimie-Technologie-4ème.pdf


Traitement de Physique-Chimie-Technologie-4ème.pdf: 47it [00:00, 88.94it/s]


Lecture du fichier : data/Physics F1-F2.pdf


Traitement de Physics F1-F2.pdf: 38it [00:01, 23.83it/s]


Lecture du fichier : data/Biology Form 3-4-5.pdf


...

Lecture du fichier : data/ECM 4ème,3ème.pdf


Traitement de ECM 4ème,3ème.pdf: 48it [00:00, 74.90it/s]


Lecture du fichier : data/ANGLAIS 4e ESG .pdf


Traitement de ANGLAIS 4e ESG .pdf: 62it [00:01, 35.75it/s]



--- Physique-Chimie-Technologie-4ème.pdf ---
Page 1 :
Page 1 sur 47      DOMAINE D’APPRENTISSAGE : SCIENCES ET TECHNOLOGIE  PROGRAMME D’ÉTUDES : PHYSIQUE-CHIMIE-TECHNOLOGIE  NIVEAU : QUATRIEME  VOLUMES HORAIRES :  VOLUME HORAIRE ANNUEL : 75 HEURES  VOLUME HORAIRE HEBDOMADAIRE : 03 HEURES  COEFFICIENT : 03


Page 2 :
Page 2 sur 47      MODULE 1 : LA MATIÈRE : SES PROPRIÉTES ET SES TRANSFORMATIONS   VOLUME HORAIRE ALLOUÉ AU MODULE : 18 HEURES   PRESENTATION DU MODULE   Ce module comporte trois (03) parties :   ➢ Les propriétés et les caractéristiques de la matière ;   ➢ Les aimants, le champ magnétique terrestre ;   ➢ La notion de réaction chimique et d’élément.   Catégories d’actions :  -Détermination des propriétés caractéristiques de la matière   -Réalisation des transformations chimiques   -Détermination des caractéristiques physiques et chimiques d’un corps     Situation problème :   Les objets qui nous entourent ont des propriétés différentes. Certains peuvent quitter de l’état li

Prenons maintenant un échantillon aléatoire des pages.

In [6]:
import random

# Sélectionne un fichier PDF particulier pour l'échantillonnage
nom_pdf_test = list(contenu_tous_les_pdfs.keys())[0]  # Exemple : le premier fichier PDF
pages_and_texts = contenu_tous_les_pdfs[nom_pdf_test]

# Vérifie si `pages_and_texts` contient au moins 3 pages
if len(pages_and_texts) >= 3:
    # Sélectionne 3 pages aléatoires
    pages_aleatoires = random.sample(pages_and_texts, k=3)
    
    # Affiche les pages sélectionnées
    for i, page in enumerate(pages_aleatoires, start=1):
        print(f"\n--- Page Aléatoire {i} ---")
        print(f"Numéro de page : {page['numero_page']}")
        print(f"Contenu : {page['texte']}\n")
else:
    print(f"Le fichier PDF '{nom_pdf_test}' contient moins de 3 pages. Impossible de faire un échantillonnage.")



--- Page Aléatoire 1 ---
Numéro de page : 30
Contenu : Page 30 sur 47    Un format se présente avec :     • Le cadre intérieur : c’est un tracé en trait de largeur 0,5mm, à10mm du bord de la feuille. Le cadre limite la zone utilisable par  le dessinateur.  • La cartouche : c’est le cadre à l’intérieur du cadre intérieur, il contient toutes les données nécessaires à l’identification et au  classement du document. Son contour est en trait fort.  Représentation du cartouche d’inscription :                          ECHELLE          NOM ET PRENOM  N°  CLASSE DATE       TITRE  ETABLISSEMENT 6  6  3  90  50  50mm 40mm


--- Page Aléatoire 2 ---
Numéro de page : 9
Contenu : Page 9 sur 47


--- Page Aléatoire 3 ---
Numéro de page : 15
Contenu : Page 15 sur 47     c-Quelle indication porte l'ampèremètre A2?   3. La tension aux bornes d’une portion de circuit   3.1. Unité et appareils de mesure ;   La tension du courant électrique se mesure à l’aide d’un Voltmètre   branché en dérivation.  L’uni

In [9]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,numero_page,nb_caracteres,nb_mots,nb_phrases,nb_tokens_estime,texte
0,1,252,46,1,63.00,Page 1 sur 47 DOMAINE D’APPRENTISSAGE : S...
1,2,1016,186,5,254.00,Page 2 sur 47 MODULE 1 : LA MATIÈRE : SES...
2,3,2255,394,17,563.75,Page 3 sur 47 2- Identifier parmi ces mots ...
3,4,2132,379,17,533.00,Page 4 sur 47 1.2- Les aimants et le champ...
4,5,1330,269,5,332.50,Page 5 sur 47 1.2.4 Utilisation d’une bouss...


### Obtenez des statistiques sur le texte

Effectuons une analyse exploratoire approximative des données (EDA) pour avoir une idée de la taille des textes (par exemple, nombre de caractères, nombre de mots, etc.) avec lesquels nous travaillons.

Les différentes tailles de textes seront un bon indicateur de la manière dont nous devrions diviser nos textes.

De nombreux modèles d'intégration ont des limites quant à la taille des textes qu'ils peuvent ingérer, par exemple le modèle [`sentence-transformers`](https://www.sbert.net/docs/pretrained_models.html) [`all-mpnet-base -v2`](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) a une taille d'entrée de 384 jetons.

Cela signifie que le modèle a été entraîné à ingérer et à transformer en textes d'intégration avec 384 jetons (1 jeton ~= 4 caractères ~= 0,75 mots).

Les textes de plus de 384 jetons codés par ce modèle seront automatiquement réduits à 384 jetons, ce qui risque de perdre certaines informations.

Nous en discuterons davantage dans la section intégration.

Pour l'instant, transformons notre liste de dictionnaires en DataFrame et explorons-la.

In [10]:
# Get stats
df.describe().round(2)

,numero_page,nb_caracteres,nb_mots,nb_phrases,nb_tokens_estime
count,47.00,47.00,47.00,47.00,47.00
mean,24.00,1177.51,222.79,9.57,294.38
std,13.71,513.32,91.16,5.09,128.33
min,1.00,13.00,4.00,1.00,3.25
25%,12.50,785.50,144.50,6.00,196.38
50%,24.00,1246.00,248.00,8.00,311.50
75%,35.50,1499.50,283.00,12.00,374.88
max,47.00,2255.00,394.00,24.00,563.75


D'accord, il semble que notre nombre moyen de jetons par page soit de 294.

### Traitement ultérieur du texte (divisation des pages en phrases)

La manière idéale de traiter le texte avant de l’intégrer reste un domaine de recherche actif.

Une méthode simple que j'ai trouvée utile consiste à diviser le texte en morceaux de phrases.

Comme dans, divisez une page de texte en groupes de 5, 7, 10 phrases ou plus (ces valeurs ne sont pas gravées dans le marbre et peuvent être explorées).

Mais nous voulons suivre le flux de travail de :

`Ingérer du texte -> le diviser en groupes/morceaux -> intégrer les groupes/morceaux -> utiliser les intégrations`

Quelques options pour diviser le texte en phrases :

1. Diviser en phrases avec des règles simples (par exemple diviser sur ". " avec `text = text.split(". ")`, comme nous l'avons fait ci-dessus).
2. Divisez en phrases avec une bibliothèque de traitement du langage naturel (NLP) telle que [spaCy](https://spacy.io/) ou [nltk](https://www.nltk.org/).

Pourquoi diviser en phrases ?

* Plus facile à gérer que des pages de texte plus volumineuses (surtout si les pages sont densément remplies de texte).
* Peut être précis et découvrir quel groupe de phrases a été utilisé pour aider dans un pipeline RAG.

> **Ressource :** Voir [instructions d'installation de spaCy](https://spacy.io/usage). 

Utilisons spaCy pour diviser notre texte en phrases, car c'est probablement un peu plus robuste que d'utiliser simplement `text.split(". ")`.

In [12]:
from spacy.lang.fr import French # see https://spacy.io/usage for install instructions

nlp = French()

# Add a sentencizer pipeline, see https://spacy.io/api/sentencizer/ 
nlp.add_pipe("sentencizer")

# Create a document instance as an example
doc = nlp("Ceci est une phrase. Voici une autre phrase.")
assert len(list(doc.sents)) == 2

# Accéder aux phrases du document
print(list(doc.sents))  # Affiche les phrases détectées


[Ceci est une phrase., Voici une autre phrase.]


Nous n'avons pas nécessairement besoin d'utiliser spaCy, cependant, il s'agit d'une bibliothèque open source conçue pour effectuer des tâches NLP comme celle-ci à grande échelle.

Alors exécutons notre petit pipeline de détermination de peine sur nos pages de texte.

In [14]:
# Divise le texte en phrases pour chaque page dans `pages_and_texts`
for item in tqdm(pages_and_texts, desc="Traitement des phrases avec SpaCy"):
    # Extraire les phrases de la page
    item["phrases"] = list(nlp(item["texte"]).sents)

    # Convertir les phrases en chaînes de caractères
    item["phrases"] = [str(phrase) for phrase in item["phrases"]]

    # Compter le nombre de phrases détectées sur la page
    item["nb_phrases_spacy"] = len(item["phrases"])

# Inspection d'une page aléatoire pour vérifier les résultats
page_aleatoire = random.sample(pages_and_texts, k=1)[0]  # Sélectionne une page aléatoire

# Afficher les résultats pour la page sélectionnée
print("\n--- Exemple de page analysée ---")
print(f"Numéro de page : {page_aleatoire['numero_page']}")
print(f"Nombre de phrases détectées (SpaCy) : {page_aleatoire['nb_phrases_spacy']}")
print("Quelques phrases extraites :")
for phrase in page_aleatoire["phrases"][:3]:  # Affiche les 3 premières phrases
    print(f"- {phrase}")

Traitement des phrases avec SpaCy: 100%|██████████| 47/47 [00:00<00:00, 47.82it/s]


--- Exemple de page analysée ---
Numéro de page : 17
Nombre de phrases détectées (SpaCy) : 17
Quelques phrases extraites :
- Page 17 sur 47    La masse d’un corps caractérise la quantité de matière que contient ou qui constitue ce corps .
-  La masse est symbolisée par la lettre « M » ou « m ».
- Elle se mesure à l’aide d’une balance et son unité légale est le kilogramme de  symbole « kg ».


Merveilleux!

Transformons maintenant la liste des dictionnaires en DataFrame et obtenons quelques statistiques.

In [15]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,numero_page,nb_caracteres,nb_mots,nb_phrases,nb_tokens_estime,nb_phrases_spacy
count,47.00,47.00,47.00,47.00,47.00,47.00
mean,24.00,1177.51,222.79,9.57,294.38,9.83
std,13.71,513.32,91.16,5.09,128.33,5.52
min,1.00,13.00,4.00,1.00,3.25,1.00
25%,12.50,785.50,144.50,6.00,196.38,6.00
50%,24.00,1246.00,248.00,8.00,311.50,8.00
75%,35.50,1499.50,283.00,12.00,374.88,13.00
max,47.00,2255.00,394.00,24.00,563.75,28.00


Maintenant que notre texte est divisé en phrases, on regroupe ces phrases ?

### Regrouper nos phrases ensemble

Faisons un pas pour diviser notre liste de phrases/texte en morceaux plus petits.

Comme vous l'avez peut-être deviné, ce processus est appelé **chunking**.

Pourquoi faisons-nous cela ?

1. Plus facile de gérer des morceaux de texte de taille similaire.
2. Ne surchargez pas la capacité des modèles d'intégration pour les jetons (par exemple, si un modèle d'intégration a une capacité de 384 jetons, il pourrait y avoir une perte d'informations si vous essayez d'intégrer une séquence de plus de 400 jetons).
3. Notre fenêtre contextuelle LLM (la quantité de jetons qu'un LLM peut accepter) peut être limitée et nécessite de la puissance de calcul, nous voulons donc nous assurer que nous l'utilisons aussi bien que possible.

Il convient de noter qu'il existe de nombreuses façons différentes de créer des morceaux d'informations/de texte.

In [16]:
# Définir la taille des groupes de phrases pour créer des chunks
taille_chunk_phrases = 10 

# Fonction pour diviser récursivement une liste en sous-listes de taille définie
def diviser_liste(liste_entree: list, 
                  taille_sous_liste: int) -> list[list[str]]:
    """
    Divise la liste_entree en sous-listes de taille taille_sous_liste (ou aussi proches que possible).

    Par exemple, une liste de 17 phrases sera divisée en deux sous-listes : [[10], [7]].
    """
    return [liste_entree[i:i + taille_sous_liste] for i in range(0, len(liste_entree), taille_sous_liste)]

# Parcourir les pages et textes pour diviser les phrases en chunks
for item in tqdm(pages_and_texts, desc="Création des chunks de phrases"):
    # Diviser les phrases en groupes
    item["chunks_phrases"] = diviser_liste(liste_entree=item["phrases"],
                                           taille_sous_liste=taille_chunk_phrases)
    
    # Ajouter le nombre total de chunks
    item["nb_chunks"] = len(item["chunks_phrases"])

# Exemple : Affiche le contenu pour une page aléatoire
import random

page_aleatoire = random.sample(pages_and_texts, k=1)[0]  # Sélectionner une page aléatoire

print("\n--- Exemple d'une page avec des chunks ---")
print(f"Numéro de page : {page_aleatoire['numero_page']}")
print(f"Nombre de chunks : {page_aleatoire['nb_chunks']}")
print("Quelques chunks de phrases :")
for i, chunk in enumerate(page_aleatoire["chunks_phrases"][:3], start=1):  # Affiche les 3 premiers chunks
    print(f"\nChunk {i} :")
    for phrase in chunk:
        print(f"- {phrase}")


Création des chunks de phrases: 100%|██████████| 47/47 [00:00<00:00, 26718.93it/s]


--- Exemple d'une page avec des chunks ---
Numéro de page : 46
Nombre de chunks : 2
Quelques chunks de phrases :

Chunk 1 :
- Page 46 sur 47    - Définition : jeu, surfaces fonctionnelles, moulage, emboutissage, forgeage, tournage, soudage, rivetage, montage à force, vis-écrou,  boulon, taraudage, filetage ;   ✓ Le moulage : Il consiste à verser une substance dans un moule, celle si prend la forme du moule après solidification exemple dans  la fabrication des parpaings.
-  ✓ Le forgeage : C’est la déformation à chaud ou à froid d’un métal sous l’action de la pression.
-  ✓ Le tournage : C’est une technique de fabrication qui consiste à faire roter la pièce pendant que l’outil se déplace en translation.
-  -  Procédé d’assemblage.
-  Assembler des pièces c’est les mettre en liaison.
- Une liaison impose une stabilité entre les surfaces en contacts des deux objets.
- Ces  surfaces de contact sont appelés surfaces fonctionnelles.
- Il existe par conséquent des assemblages démontables com

In [17]:
# Sélectionner un exemple aléatoire parmi les pages traitées
import random

# Sélectionne une page aléatoire de `pages_and_texts`
page_aleatoire = random.sample(pages_and_texts, k=1)[0]

# Afficher des informations sur la page sélectionnée
print("\n--- Exemple de page avec des chunks ---")
print(f"Numéro de page : {page_aleatoire['numero_page']}")
print(f"Nombre de chunks de phrases : {page_aleatoire['nb_chunks']}")

# Affiche les chunks (chaque chunk contient plusieurs phrases, si la page a plus de 10 phrases)
print("Quelques chunks de phrases :")
for i, chunk in enumerate(page_aleatoire["chunks_phrases"][:3], start=1):  # Affiche les 3 premiers chunks
    print(f"\nChunk {i} :")
    for phrase in chunk:
        print(f"- {phrase}")

# Exemple de sélection d'un chunk aléatoire parmi ceux de cette page
if page_aleatoire["nb_chunks"] > 1:  # Vérifie si la page contient plusieurs chunks
    chunk_aleatoire = random.choice(page_aleatoire["chunks_phrases"])
    print("\n--- Chunk aléatoire sélectionné ---")
    for phrase in chunk_aleatoire:
        print(f"- {phrase}")
else:
    print("\nLa page ne contient qu'un seul chunk.")



--- Exemple de page avec des chunks ---
Numéro de page : 4
Nombre de chunks de phrases : 2
Quelques chunks de phrases :

Chunk 1 :
- Page 4 sur 47    1.2- Les aimants  et le champ magnétique terrestre   Un aimant est un matériau développant naturellement un champ magnétique et capable d’attirer le fer, le nickel, le cobalt et le chrome.
-  Situation problème : Djiaha rapproche 2 aimants et constate qu’ils se collent.
- Ensuite elle retourne un coté d’un aimant et les  rapprochent.
- Elle constate qu’ils ne veulent pas se coller.
-  Expliquer lui ce qui se passe.
-  1.2.1 Les pôles d’un aimant   Un aimant à un pôle nord et un pôle sud quel qu’en soit sa taille.
- Les pôles de même nature se repoussent et celle de natures contraires  s’attirent.
- Il existe des aimants naturels (la magnétite et la terre) et des aimants artificiels (les aimants permanents tels que les aimants en  U, en barreau aimanté, en cercle, l’aiguille magnétique et les aimants temporaires tels que les électroaimant

In [18]:
# Crée un DataFrame à partir des données de `pages_and_texts`
df = pd.DataFrame(pages_and_texts)

# Affiche les statistiques descriptives
df_stats = df.describe().round(2)

# Affiche le DataFrame avec les statistiques
print(df_stats)

       numero_page  nb_caracteres  nb_mots  nb_phrases  nb_tokens_estime  \
count        47.00          47.00    47.00       47.00             47.00   
mean         24.00        1177.51   222.79        9.57            294.38   
std          13.71         513.32    91.16        5.09            128.33   
min           1.00          13.00     4.00        1.00              3.25   
25%          12.50         785.50   144.50        6.00            196.38   
50%          24.00        1246.00   248.00        8.00            311.50   
75%          35.50        1499.50   283.00       12.00            374.88   
max          47.00        2255.00   394.00       24.00            563.75   

       nb_phrases_spacy  nb_chunks  
count             47.00      47.00  
mean               9.83       1.38  
std                5.52       0.53  
min                1.00       1.00  
25%                6.00       1.00  
50%                8.00       1.00  
75%               13.00       2.00  
max               2

### Diviser chaque morceau en son propre élément

Nous aimerions embedder chaque morceau de phrases dans sa propre représentation numérique.

Donc, pour garder les choses claires, créons une nouvelle liste de dictionnaires contenant chacun un seul morceau de phrases avec des informations relatives telles que le numéro de page ainsi que des statistiques sur chaque morceau.

In [21]:
import re

# Liste pour stocker les chunks traités
pages_et_chunks = []

# Parcourt les pages et les chunks pour créer des données sur chaque chunk
for item in tqdm(pages_and_texts, desc="Traitement des chunks de phrases"):
    for chunk_de_phrases in item["chunks_phrases"]:
        chunk_dict = {}
        chunk_dict["numero_page"] = item["numero_page"]
        
        # Joindre les phrases pour former un "paragraphe" ou chunk
        chunk_texte = "".join(chunk_de_phrases).replace("  ", " ").strip()
        chunk_texte = re.sub(r'\.([A-Z])', r'. \1', chunk_texte)  # Format ".A" -> ". A" pour toute combinaison point/majuscule
        chunk_dict["chunk_texte"] = chunk_texte

        # Calculer les statistiques pour le chunk
        chunk_dict["chunk_nb_caracteres"] = len(chunk_texte)
        chunk_dict["chunk_nb_mots"] = len(chunk_texte.split(" "))
        chunk_dict["chunk_nb_tokens"] = len(chunk_texte) / 4  # Estimation des tokens (1 token ≈ 4 caractères)
        
        # Ajouter le chunk traité à la liste des chunks
        pages_et_chunks.append(chunk_dict)

# Vérifie combien de chunks ont été traités
print(f"Nombre total de chunks traités : {len(pages_et_chunks)}")

Traitement des chunks de phrases: 100%|██████████| 47/47 [00:00<00:00, 428.59it/s]

Nombre total de chunks traités : 65


In [22]:
# View a random sample
random.sample(pages_et_chunks, k=1)


[{'numero_page': 22,
  'chunk_texte': "Page 22 sur 47  b- Après plusieurs pluies, on a constaté que le lessivage de son champs a entrainer la croissance anormale des plantes aquatiques et la mort des animaux aquatiques due à l’utilisation abusive des engrais dans son champs. Comment explique-t-on ce phénomène ?les engrais sont-ils dangereux pour notre santé ? 3.2. Les engrais  3.2 .1. Définition : Fertilisation, élément fertilisant, engrais ;  La fertilisation est le processus consistant à apporter à un milieu de culture, tel que le sol, les éléments minéraux nécessaires au développement de la plante. Les engrais sont des substances organiques ou minérales, souvent utilisées en mélanges, destinées à apporter aux plantes des compléments d'éléments nutritifs, de façon à améliorer leur croissance, et à augmenter le rendement et la qualité des cultures. Les éléments fertilisants : produits destinés à assurer ou à améliorer la nutrition des végétaux et les propriétés des sols. 3.2.2. Les él

**Excellent**!


### Embedder  nos morceaux de texte

In [23]:
# Créer une liste des chunks de texte
chunks_texte = [item["chunk_texte"] for item in pages_et_chunks]


Traitement des chunks:   0%|          | 0/2 [00:00<?, ?it/s]

Traitement des chunks: 100%|██████████| 2/2 [00:03<00:00,  1.90s/it]

                             chunk_texte  embedding
0          Ceci est un exemple de texte.  -0.357687
1  Un autre exemple de morceau de texte.  -0.148989
